In [1]:
from utils.df_handle import *

In [6]:
import time
from functools import wraps
from memory_profiler import memory_usage

def profile(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        fn_kwargs_str = ', '.join(f'{k}={v}' for k, v in kwargs.items())
        print(f'\n{fn.__name__}({fn_kwargs_str})')

        # Measure time
        t = time.perf_counter()
        retval = fn(*args, **kwargs)
        elapsed = time.perf_counter() - t
        print(f'Time   {elapsed:0.4}')

        # Measure memory
        mem, retval = memory_usage((fn, args, kwargs), retval=True, timeout=200, interval=1e-7)

        print(f'Memory {max(mem) - min(mem)}')
        return retval

    return inner

In [2]:
@profile
def work(n):
    for i in range(n):
        2 ** n

In [3]:
work(10)


work()
Time   7.1e-06
Memory 0.05078125


In [4]:
work(n=10000)


work(n=10000)
Time   0.2083
Memory 0.04296875


In [5]:
beer=['a','b']

In [16]:
def add(x, y):
    return x+y

c = [1, 5]

print(add(**c))

TypeError: __main__.add() argument after ** must be a mapping, not list

In [9]:
from typing import Iterator, Dict, Any
from urllib.parse import urlencode
import requests


def iter_beers_from_api(page_size: int = 5) -> Iterator[Dict[str, Any]]:
    session = requests.Session()
    page = 1
    while True:
        response = session.get('https://api.punkapi.com/v2/beers?' + urlencode({
            'page': page,
            'per_page': page_size
        }))
        response.raise_for_status()

        data = response.json()
        if not data:
            break

        yield from data

        page += 1

In [10]:
beers = iter_beers_from_api()

In [12]:
def create_staging_table(cursor) -> None:
    cursor.execute("""
        DROP TABLE IF EXISTS staging_beers;
        CREATE UNLOGGED TABLE staging_beers (
            id                  INTEGER,
            name                TEXT,
            tagline             TEXT,
            first_brewed        DATE,
            description         TEXT,
            image_url           TEXT,
            abv                 DECIMAL,
            ibu                 DECIMAL,
            target_fg           DECIMAL,
            target_og           DECIMAL,
            ebc                 DECIMAL,
            srm                 DECIMAL,
            ph                  DECIMAL,
            attenuation_level   DECIMAL,
            brewers_tips        TEXT,
            contributed_by      TEXT,
            volume              INTEGER
        );
    """)

In [13]:
beers = list(iter_beers_from_api()) * 100

In [20]:
type(beers)

list

In [2]:
df1 = get_ps_df("""select * from test_dataframe""")

In [2]:
from datetime import datetime
import pandas as pd
import numpy as np
import psycopg2.extras as extras
from psycopg2 import OperationalError, errorcodes, errors
import psycopg2
from utils.df_handle import *

In [6]:
startdate1=datetime(2021, 5, 5)
startdate2=datetime(2021, 5, 5)
startdate3=datetime(2021, 5, 7)
data = {'date':[startdate1, startdate2, startdate3], 'marks':[5,6,7], 'name':['duy','khanh','vinh'],'bonus':[1.3, 2.3, 3.3]}
df1=pd.DataFrame(data)

In [7]:
df1

,date,marks,name,bonus
0,2021-05-05,5,duy,1.3
1,2021-05-05,6,khanh,2.3
2,2021-05-07,7,vinh,3.3


In [8]:
rows = [tuple(x) for x in df1.to_numpy()]

In [15]:

insert_rows_psql(table='test_dataframe', rows=rows, replace=True, target_fields=[x.lower() for x in df1.columns], replace_index=['marks','name'])


insert_rows_psql(table=test_dataframe, rows=[(Timestamp('2021-05-05 00:00:00'), 5, 'duy', 1.5), (Timestamp('2021-05-05 00:00:00'), 6, 'khanh', 2.5), (NaT, 7, 'vinh', 3.5)], replace=True, target_fields=['date', 'marks', 'name', 'bonus'], replace_index=['marks', 'name'])
INSERT INTO test_dataframe (date, marks, name, bonus) VALUES (%s,%s,%s,%s) ON CONFLICT (marks, name) DO UPDATE SET date = excluded.date, bonus = excluded.bonus
INSERT INTO test_dataframe (date, marks, name, bonus) VALUES (%s,%s,%s,%s) ON CONFLICT (marks, name) DO UPDATE SET date = excluded.date, bonus = excluded.bonus
INSERT INTO test_dataframe (date, marks, name, bonus) VALUES (%s,%s,%s,%s) ON CONFLICT (marks, name) DO UPDATE SET date = excluded.date, bonus = excluded.bonus


InvalidDatetimeFormat: invalid input syntax for type timestamp: "NaT"
LINE 1: ...test_dataframe (date, marks, name, bonus) VALUES ('NaT'::tim...
                                                             ^


In [38]:
# def tuples(df):
#     tuples = list(df.itertuples(index=False, name=None))
#     return tuples

In [39]:
# tuples = tuples(df1)

In [40]:
tuples

[(Timestamp('2021-05-01 00:00:00'), 5, 'duy', 1.7),
 (Timestamp('2021-05-02 00:00:00'), 6, 'khanh', 2.7),
 (Timestamp('2021-05-03 00:00:00'), 7, 'vinh', 3.7),
 (Timestamp('2021-05-07 18:08:51.057000'), 0, 'MR0001', 35238.094)]

In [42]:
cursor = get_cur()

In [6]:
tpls

[(Timestamp('2021-05-01 00:00:00'), 5, 'duy', 1.7),
 (Timestamp('2021-05-02 00:00:00'), 6, 'khanh', 2.7),
 (Timestamp('2021-05-03 00:00:00'), 7, 'vinh', 3.7)]

In [29]:
# Define function using psycopg2.extras.execute_values() to insert the dataframe.
def execute_values_upsert(conn, datafrm, table, pk):
    # Creating a list of tupples from the dataframe values
    tpls = [tuple(x) for x in datafrm.to_numpy()]
    # dataframe columns with Comma-separated
    cols = ','.join(list(datafrm.columns)) 
    # SQL query to execute
    sql = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    replace_target = ["{0} = excluded.{0}".format(col) for col in datafrm.columns if col not in pk]
    strr = f" ON CONFLICT ({', '.join(['marks','name'])}) DO UPDATE SET {', '.join(replace_target)}"
    sql += strr
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, sql, tpls, page_size=1000)
        conn.commit()
        print(sql)
        print("Data inserted using execute_values() successfully..")
    except (Exception, psycopg2.DatabaseError) as err:
        print(sql)
        print(err)
        cursor.close()

In [31]:
conn = get_pg2_conn()

In [ ]:
execute_values()

In [21]:
from psycopg2.extras import execute_values

In [ ]:
execute_values()

In [14]:
tpls = [tuple(x) for x in df1.to_numpy()]

In [16]:
replace_target = ["{0} = excluded.{0}".format(col) for col in ['date', 'bonus']]

In [17]:
replace_target

['date = excluded.date', 'bonus = excluded.bonus']

In [18]:
strr = f" ON CONFLICT ({', '.join(['marks','name'])}) DO UPDATE SET {', '.join(replace_target)}"

In [19]:
strr

' ON CONFLICT (marks, name) DO UPDATE SET date = excluded.date, bonus = excluded.bonus'

In [15]:
len(tpls)

3

In [32]:
execute_values_upsert(conn, df1, 'test_dataframe', pk=['marks','name'])

INSERT INTO test_dataframe(date,marks,name,bonus) VALUES %s ON CONFLICT (marks, name) DO UPDATE SET date = excluded.date, bonus = excluded.bonus
Data inserted using execute_values() successfully..
